<a href="https://colab.research.google.com/github/Sug-ar-N-Spice/Dr.Chats/blob/Patricia/Patricia_Dr_chat_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence-transformers
! pip install sacremoses
! pip install transformers
! pip install datasets
! pip install torch
#!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
from typing import List, Dict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer


In [4]:

##STOP WORDS IN NLP DONT MEAN ANYTHING LIKE WE THEY THEY JUST COMPLETE THE SENTENCE

## THIS IS CLASS THAT Cleans the data
class MEDDataPreprocessor:
    """
    Preprocessor for general medical data.
    This class handles cleaning, normalization, and preparation of text data
    related to medical topics for use in a medical chatbot.
    """

    def __init__(self):
        """Initialize the preprocessor with necessary NLTK downloads."""
        nltk.download('punkt', quiet=True)
        nltk.download('stopwords', quiet=True)
        self.stop_words = set(stopwords.words('english'))

        # Minimal list of medical terms to preserve
        self.medical_terms = {
            # Terms conflicting with stopwords
            'a', 'am', 'an', 'as', 'at', 'be', 'by', 'in', 'no', 'on', 'or', 'to', 'up',
            # Critical abbreviations to always preserve
            'ct', 'dr', 'er', 'hiv', 'hr', 'icu', 'iv', 'mr', 'ms'
        }

        self.stop_words = self.stop_words - self.medical_terms

    def clean_text(self, text: str) -> str:



        """
        Clean and normalize the input text.

        Args:
            text (str): Raw input text

        Returns:
            str: Cleaned and normalized text
        """

        if pd.isna(text):
            return ""

        # Convert to lowercase
        text = text.lower()

        # Remove special characters but keep medical symbols
        text = re.sub(r'[^a-zA-Z0-9\s+\-/%]', '', text)

        # Remove extra whitespace
        text = ' '.join(text.split())

        return text

    def remove_stopwords(self, text: str) -> str:
        """
        Remove stopwords from the text, keeping medical specific terms.

        Args:
            text (str): Input text

        Returns:
            str: Text with stopwords removed
        """
        words = word_tokenize(text) ### this is resulting in a list of words was converting sentence / paragraph into a list of words

        filtered_words = [word for word in words if word.lower() not in self.stop_words]
        return ' '.join(filtered_words)

    def preprocess_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocess the entire dataframe.

        Args:
            df (pd.DataFrame): Input dataframe

        Returns:
            pd.DataFrame: Preprocessed dataframe
        """
        # change columns names depending which csv file you are using cleaning text and removing stopwords
        df['clean_question'] = df['question'].apply(self.clean_text).apply(self.remove_stopwords)
        df['clean_context'] = df['context'].apply(self.clean_text).apply(self.remove_stopwords)

        # Combine cleaned abstract and results or full_texts depending on which csv you are using
        df['combined_text'] = df['clean_question'] + ' ' + df['clean_context']

        return df

    def prepare_for_model(self, text: str, max_length: int = 512) -> str: ##looks at paragraph, cuts the paragraph if more than 512 This takes the sentence splits to words and has a max length
        """
        Prepare text for model input, truncating if necessary.

        Args:
            text (str): Input text
            max_length (int): Maximum number of words

        Returns:
            str: Prepared text
        """
        words = text.split()
        if len(words) > max_length:
            return ' '.join(words[:max_length])
        return text



In [5]:
def preprocess_dataframe(self, df): ###USUALLY YOU SEE SELF IN A CLASS This allows you to code attributes in a class
    """
    Preprocess the entire dataframe.

    Args:
        df (pd.DataFrame or DatasetDict): Input dataframe with 'question' and 'context' columns

    Returns:
        pd.DataFrame: Preprocessed dataframe
    """

    # Check if df is a DatasetDict object-- When we try to find dataset Dict- that means we havent converted it to pandas and cleaning whole dataset


    if isinstance(df.DatasetDict):
        # Process each split separately and combine into a DataFrame
        all_data = []
        for split in df:
                    # Assuming all splits have the same columns
                    # Process the columns in each split
            df_split = df[split].to_pandas()  # Convert to DataFrame

            df_split['clean_question'] = df_split['question'].apply(self.clean_text).apply(self.remove_stopwords)
            df_split['clean_context'] = df_split['context'].apply(self.clean_text).apply(self.remove_stopwords)
            df_split['combined_text'] = df_split['clean_question'] + ' ' + df_split['clean_context']

            all_data.extend(df_split.to_dict('records')) # Add processed data to the list

        processed_df= pd.DataFrame(all_data) # Create a new DataFrame from the combined data

    else:  # If it's a regular DataFrame, process as before
        processed_df= df.copy()  # Create a copy to avoid modifying the original DataFrame
        processed_df['clean_question'] = processed_df['question'].apply(self.clean_text).apply(self.remove_stopwords)
        processed_df['clean_context'] = processed_df['context'].apply(self.clean_text).apply(self.remove_stopwords)
        processed_df['combined_text'] = processed_df['clean_question'] + ' ' + processed_df['clean_context']

    return processed_df  # Return the processed data

In [18]:
class SlidingWindowSelector:
    def __init__(self, model_name='all-MiniLM-L6-v2', window_size=500, stride=250, batch_size=16):
        self.model = SentenceTransformer(model_name)
        self.contexts = []
        self.embeddings = []
        self.window_size = window_size
        self.stride = stride
        self.batch_size = batch_size

    def fit(self, contexts):
        for i in range(0, len(contexts), self.stride):
            window = contexts[i:i+self.window_size]
            self.contexts.extend(window)

            # Process embeddings in batches
            for j in range(0, len(window), self.batch_size):
                batch = window[j:j+self.batch_size]
                batch_embeddings = self.model.encode(batch)
                self.embeddings.extend(batch_embeddings)

        self.embeddings = np.array(self.embeddings)

    def update(self, new_contexts):
        new_embeddings = []
        for i in range(0, len(new_contexts), self.batch_size):
            batch = new_contexts[i:i+self.batch_size]
            batch_embeddings = self.model.encode(batch)
            new_embeddings.extend(batch_embeddings)

        self.contexts.extend(new_contexts)
        self.embeddings = np.vstack([self.embeddings, np.array(new_embeddings)])

    def get_most_relevant_context(self, question, top_n=3):
        question_embedding = self.model.encode([question])

        # Calculate similarities in batches
        all_similarities = []
        for i in range(0, len(self.embeddings), self.batch_size):
            batch_embeddings = self.embeddings[i:i+self.batch_size]
            batch_similarities = np.dot(batch_embeddings, question_embedding.T).squeeze()
            all_similarities.extend(batch_similarities)

        top_indices = np.argsort(all_similarities)[-top_n:][::-1]
        return [self.contexts[i] for i in top_indices]


In [19]:
class DRChatbot:
    def __init__(self, qa_model="microsoft/BioGPT", window_size=500, stride=250, batch_size=16):
        self.preprocessor = MEDDataPreprocessor()
        self.qa_tokenizer = AutoTokenizer.from_pretrained(qa_model)
        self.qa_model = AutoModelForCausalLM.from_pretrained(qa_model)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.qa_model.to(self.device)
        self.context_selector = SlidingWindowSelector(window_size=window_size, stride=stride, batch_size=batch_size)

    def preprocess_data(self, df):
        processed_df = self.preprocessor.preprocess_dataframe(df)
        return processed_df['combined_text'].tolist()

    def fit_context_selector(self, all_contexts):
        self.context_selector.fit(all_contexts)

    def update_context_selector(self, new_contexts):
        self.context_selector.update(new_contexts)

    def answer_question(self, question):
        relevant_contexts = self.context_selector.get_most_relevant_context(question, top_n=3)
        combined_context = " ".join(relevant_contexts)

        prompt = f"Based on the following medical information:\n{combined_context}\n\nQuestion: {question}\nAnswer:"
        inputs = self.qa_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.device)

        input_length = inputs["input_ids"].shape[1]
        max_new_tokens = min(100, 1024 - input_length)

        with torch.no_grad():
            output = self.qa_model.generate(
                inputs["input_ids"],
                max_new_tokens=max_new_tokens,
                num_return_sequences=1,
                do_sample=True,
                temperature=0.7
            )

        return self.qa_tokenizer.decode(output[0], skip_special_tokens=True)



In [23]:
def process_large_csv(file_path, window_size=500, stride=250, batch_size=16, chunk_size=10000):
    chatbot = DRChatbot(window_size=window_size, stride=stride, batch_size=batch_size)
    all_contexts = []

     # Process the CSV file in chunks
    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
        contexts = chatbot.preprocess_data(chunk)
        if i == 0:
            chatbot.fit_context_selector(contexts)
        else:
            chatbot.update_context_selector(contexts)
        print(f"Processed chunk {i+1}, total contexts: {len(chatbot.context_selector.contexts)}")

    return chatbot



In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Example usage
file_path = '/content/drive/My Drive/million_sample.csv'
chatbot = process_large_csv(file_path, window_size=500, stride=250, batch_size=16)

Processed chunk 1, total contexts: 19750
Processed chunk 2, total contexts: 29750
Processed chunk 3, total contexts: 39750
Processed chunk 4, total contexts: 49750
Processed chunk 5, total contexts: 59750
Processed chunk 6, total contexts: 69750
Processed chunk 7, total contexts: 79750
Processed chunk 8, total contexts: 89750
Processed chunk 9, total contexts: 99750
Processed chunk 10, total contexts: 109750
Processed chunk 11, total contexts: 119750
Processed chunk 12, total contexts: 129750
Processed chunk 13, total contexts: 139750


In [ ]:


# Ask a question
question = "What are the symptoms of asthma?"
answer = chatbot.answer_question(question)
print(f"Q: {question}\nA: {answer}")

In [ ]:
# Process the large CSV file
chatbot = process_large_csv('million_sample.csv')


In [ ]:
# Example usage
question = "what are the symptons of asthma?"
answer = chatbot.answer_question(question)
print(f"Q: {question}\nA: {answer}")

Q: what are the symptons of asthma?
A: Based on the following medical information: name group a member asthma asthma impact asthma exacerbations asthma impact asthma exacerbations asthma triggers on asthma-related quality life in patients severe or difficult-to-treat asthma goal national asthma guidelines background maintenance asthma control time a clear goal national asthma guidelines studies addressed natural history asthma control time objective to assess impairment domain asthma control time in patients persistent asthma to determine predictors consequences controlled uncontrolled asthma time methods patients 18-56 years old persistent asthma completed baseline november 2007 follow-up asthma surveys april july october 2008 included in study survey included asthma control test as well as questions regarding patient asthma characteristics health care utilization pharmacy exacerbations 2008 obtained administrative data results baseline first follow-up surveys completed by 1267 patien